import dependencies

In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

: 

In [17]:
import sys
sys.path.append("/Users/emiliekibsgaard/Desktop/MetaprivBIDS-Assessment")  
import metapriv_stat as ST
from metapriv_stat import plot_calc
from metapriv_stat import rst_outlier_case2

import participants.tsv from OpenNeuro, read and export as csv for later use

In [5]:
AOMIC = 'https://s3.amazonaws.com/openneuro.org/ds003097/participants.tsv?versionId=9gylMj_lF7dOQFiiUSuQ4OeHtgxzGUtv'

AOMIC = pd.read_csv(AOMIC, sep='\t')
#AOMIC.replace(np.nan, -999, inplace=True)
AOMIC.to_csv('data_ds003097/AOMIC_dataset_ds003097.csv', sep=',', index=False)

check missing data

In [ ]:
na_percentage = AOMIC.isna().mean() * 100
na_percentage = na_percentage[na_percentage >= 10].sort_values(ascending=False)
na_percentage

Compute privacy scores and store them in a csv files for later use

re-read the csv files -- allows bypassing the previous steps

In [9]:
suda = pd.read_csv('data_ds003097/SUDA_ds003097.csv', sep=',')
pif = pd.read_csv('data_ds003097/PIF_ds003097.csv', sep=',',index_col='Unnamed: 0').sort_index()
suda_field_score =  pd.read_csv('data_ds003097/att_suda.csv', sep=',').drop(columns=['Unnamed: 0'], errors='ignore')
k_combined_field =  pd.read_csv('data_ds003097/normalized_difference_results.csv', sep=',').drop(columns=['Difference in Unique Rows','Unique Rows After Removal']).rename(columns={'Column': 'variable'})
k_combined_all = pd.read_csv('data_ds003097/k-combined-all-2-11.csv', sep=',')
k_combined_field_non_risk = pd.read_csv('data_ds003097/normalized_difference_results_non.csv', sep=',').drop(columns=['Difference in Unique Rows','Unique Rows After Removal']).rename(columns={'Column': 'variable'})

In [ ]:
AOMIC

In [ ]:
ST.stats(suda, pif, suda_field_score, k_combined_field)

# WHOLE DATASET RISK VALUE SUM

In [ ]:
sum_score_df  = ST.calculate_summed_dis_scores(k_combined_all, AOMIC, sample_fraction=0.3, missing_value= np.nan)

In [ ]:
plot_calc(sum_score_df)

## T-test between risky and non-risky k-global values for variables. 

### Non risky variables

In [ ]:
k_combined_field_non_risk

### Risk variables 

In [ ]:
k_combined_field

## Test

In [ ]:
mean_risk_values = (k_combined_field['Normalized Difference']).mean()
print('Mean, risk identifiers:',mean_risk_values)

In [ ]:
mean_risk_values = (k_combined_field_non_risk['Normalized Difference']).mean()
print('Mean, non-risk identifiers:',mean_risk_values)

In [ ]:
t, p = stats.ttest_ind(k_combined_field['Normalized Difference'], k_combined_field_non_risk['Normalized Difference'])
print('t:',t,'p-value:', p)

### PIF & SUDA Outliers 

In [18]:
#SUDA
class_outliers, madn, mad, outlier_indices, above_outlier_indices = ST.rst_outlier_case2(suda, 'dis-score')

In [ ]:
print(len(above_outlier_indices))

In [ ]:
#PIF
class_outliers, madn, mad, outlier_indices, above_outlier_indices = rst_outlier_case2(pif, 'RIG')
print(len(above_outlier_indices))

In [ ]:
AOMIC.loc[338]

## Improving Risk

In [22]:
suda_improved = pd.read_csv('data_ds003097/improved_suda.csv', sep=',')
pif_improved = pd.read_csv('data_ds003097/improved_PIF.csv', sep=',',index_col='Unnamed: 0').sort_index()

In [ ]:
class_outliers, madn, mad, outlier_indices, above_outlier_indices = ST.rst_outlier_case2(suda_improved, 'dis-score')
print(len(above_outlier_indices))

In [ ]:
class_outliers, madn, mad, outlier_indices, above_outlier_indices = rst_outlier_case2(pif_improved, 'RIG')
print(len(above_outlier_indices))